In [ ]:
import requests
from 

HEADERS = {"Authorization": f"Bearer {GITHUB_TOKEN}"}
API_URL = "https://api.github.com/graphql"

query = """
query ($after: String) {
  search(query: "stars:>300", type: REPOSITORY, first: 50, after: $after) {
    repositoryCount
    pageInfo {
      endCursor
      hasNextPage
    }
    edges {
      node {
        ... on Repository {
          nameWithOwner
          description
          stargazerCount
          forkCount
          url
          updatedAt
        }
      }
    }
  }
}
"""

def fetch_repos():
    repos = []
    variables = {"after": None}
    while True:
        response = requests.post(API_URL, json={"query": query, "variables": variables}, headers=HEADERS)
        data = response.json()

        if "errors" in data:
            print("Error:", data["errors"])
            break

        search_data = data["data"]["search"]
        for edge in search_data["edges"]:
            repos.append(edge["node"])

        if not search_data["pageInfo"]["hasNextPage"]:
            break

        variables["after"] = search_data["pageInfo"]["endCursor"]

    return repos

if __name__ == "__main__":
    all_repos = fetch_repos()
    print(f"Found {len(all_repos)} repositories with >300 stars.")
    for repo in all_repos[:5]:
        print(f"{repo['nameWithOwner']} — ⭐ {repo['stargazerCount']} — {repo['url']}")
